In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import logging

for handler in logging.root.handlers:
    logging.root.removeHandler(handler)
logging.basicConfig(level=logging.INFO)

In [13]:
import os

from feast import Client, Feature, Entity, ValueType, FeatureTable
from feast.data_source import FileSource, KafkaSource
from feast.data_format import ParquetFormat, AvroFormat

In [15]:
client = Client(core_url="localhost:8090", serving_url='localhost:8092')
client.list_projects()

['default']

In [ ]:
driver_id = Entity(name="driver_id", description="Driver identifier", value_type=ValueType.INT64)